# CHEM 571B Fall 2019 Exam

1. Demonstrate that for symmetric diatomic two-electron systems, such as the H$_{2}$ molecule, in a minimal basis (e.g. STO-3G) the optimal basis coefficient matrix in the Hartree-Fock procedure is given as
$\mathbf{C} = \begin{bmatrix} (2(1+S_{12})^{-0.5} & (2(1-S_{12})^{-0.5} \\ (2(1+S_{12})^{-0.5} & -(2(1+S_{12})^{-0.5}\end{bmatrix}$
where $S_{12} = \langle \phi_1|\phi_2 \rangle$ and $\phi_i$ is the $i$th atomic basis function.

2. Show that the full CI Hamiltonian matrix for a H$_2$ system in a minimal basis is given as
$\mathbf{H_{CI}} = \begin{bmatrix} 2h_{11} + J_{11} & K_{12} \\ K_{12} & 2h_{22} + J_{22}\end{bmatrix}$, where $h_{11} = \langle \psi_1|h(1)|\psi_1\rangle$, $J_{ii} = \langle \psi_i\psi_i|r_{12}^{-1}|\psi_i\psi_i \rangle$, $K_{ij} = \langle \psi_i\psi_i|r_{12}^{-1}|\psi_j\psi_j\rangle$ and $\psi_i$ is the $i$th molecular spacial orbital.

3. Compute the dissocation curve for the Li$_{2}$ molecule in a minimal STO-3G (alpha and d values for the Li 1s and Li 2s given below) basis at the following levels of theory

    a. rHF
    
    b. Configuration Interaction in which you only consider two determinants: the ground-state and the double excited state $\sigma^2 \rightarrow \sigma^{*2}$.
    
    c. What is the minimum energy bond distance and dissociation energy in both rHF and CI?
    
    d. What is the physical nature of the bond? (i.e. is it a typical covalent bond?)
    
    e. Plot the electron density as a function of position along the bond vector. (only for rHF)

In [1]:
# The following pieces of code may be useful for problem 3

# set STO-3G basis 
alpha1s = [0.15432897,0.53532814,0.44463454]
alpha2s = [-0.09996723,0.39951283,0.70011547]
d1s = [16.1195750,2.9362007,0.7946505]
d2s = [0.6362897,0.1478601,0.0480887]


# orthogonalize basis for arbitrary number of basis functions
def orthogonalize_basis(S):
    M = S.shape[1]
    e, v = np.linalg.eig(S)
    s = np.diag(e**-0.5)
    X = np.dot(v,s)
    return X

# create and populate density matrix given C and N (number of electrons)
def constructDensityMat(C,N):
    M = C.shape[0]
    P = np.zeros((M,M),dtype=float)
    for i in range(M):
        for j in range(i,M):
            for a in range(N//2):
                P[i,j] += C[i,a]*C[j,a]
            P[i,j] *= 2.0
            P[j,i] = P[i,j]
    return P

# compute one electron potential energy integral for gaussian basis function
def gaussian_potential(alpha, RA, beta, RB, RC):
    AplusB = alpha + beta
    RAminusRB = RA - RB
    RARB2 = np.dot(RAminusRB,RAminusRB)
    RP = (alpha*RA+beta*RB)/AplusB
    RPminusRC = RP - RC
    RPRC2 = np.dot(RPminusRC,RPminusRC)
    if (RPRC2 == 0.0):
        return 2.0*np.pi/AplusB
    else:
        t = np.sqrt(AplusB*RPRC2)
        prefactor = 2.0*np.pi*0.5/AplusB*np.sqrt(np.pi)/t*erf(t)
        return prefactor*np.exp(-alpha*beta/AplusB*RARB2)